In [1]:
from ortools.sat.python import cp_model
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [12]:
grid = [[-2, 2, 1, 1, 1, -2, 2],
       [-2, 2, 1, 1, 1, -2, 2],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [-2, 2, 1, 1, 1, -2, 2],
       [-2, 2, 1, 1, 1, -2, 2]]
grid

[[-2, 2, 1, 1, 1, -2, 2],
 [-2, 2, 1, 1, 1, -2, 2],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [-2, 2, 1, 1, 1, -2, 2],
 [-2, 2, 1, 1, 1, -2, 2]]

In [95]:
# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

n = range(7)
S = range(7**2)

x = {(i,j,s): model.NewIntVar(-2 , 2, f'x_{i}_{j}_{s}') for i in n for j in n for s in S}

up = {(i,j,s): model.NewBoolVar(f'up_{i}_{j}_{s}') for i in n for j in n for s in S}
down = {(i,j,s): model.NewBoolVar(f'down_{i}_{j}_{s}') for i in n for j in n for s in S}
left = {(i,j,s): model.NewBoolVar(f'left_{i}_{j}_{s}') for i in n for j in n for s in S}
right = {(i,j,s): model.NewBoolVar(f'right_{i}_{j}_{s}') for i in n for j in n for s in S}
none = {(i,j,s): model.NewBoolVar(f'none_{i}_{j}_{s}') for i in n for j in n for s in S}

term = {(s): model.NewBoolVar(f'term_{s}') for s in S}

for i in n:
    for j in n:
        model.Add(x[i,j,0] == grid[i][j])

model.AddAtLeastOne([term[s] for s in S])

for s in S:
    model.AddAtMostOne([up[i,j,s] for i in n for j in n])
    model.AddAtMostOne([down[i,j,s] for i in n for j in n])
    model.AddAtMostOne([left[i,j,s] for i in n for j in n])
    model.AddAtMostOne([right[i,j,s] for i in n for j in n])

    model.Add(sum(up[i,j,s] + down[i,j,s] + left[i,j,s] + right[i,j,s] for i in n for j in n) <= 1)
    
    model.Add(sum(x[i,j,s] for i in n for j in n) == 1).OnlyEnforceIf(term[s])
    #model.Add(x[3,3,s] == 1).OnlyEnforceIf(term[s])
    model.Add(sum(x[i,j,s] for i in n for j in n) != 1).OnlyEnforceIf(term[s].Not())

    if s > 0:
        if s != S:
            model.Add(term[s-1] <= term[s])
        for i in n:
            for j in n:
                # No choice made
                model.Add(up[i,j,s] + down[i,j,s] + left[i,j,s] + right[i,j,s] == 0).OnlyEnforceIf(none[i,j,s])
                model.Add(up[i,j,s] + down[i,j,s] + left[i,j,s] + right[i,j,s] == 1).OnlyEnforceIf(none[i,j,s].Not())

                # Board stays the same if no choice made
                model.Add(x[i,j,s-1] == x[i,j,s]).OnlyEnforceIf(none[i,j,s])

                if grid[i][j] in [2,-2]:
                    model.Add(up[i,j,s] + down[i,j,s] + left[i,j,s] + right[i,j,s] == 0)  
                
                # Up
                if i >= 2 and any([i not in [2,3],1<j<5]):
                    # If up, then this value is == 1
                    model.Add(x[i,j,s-1] == 1).OnlyEnforceIf(up[i,j,s])  
                    # If up, then the value 1 spot above == 1
                    model.Add(x[i-1,j,s-1] == 1).OnlyEnforceIf(up[i,j,s])                    
                    # If up, then the value 2 spots above == 0
                    model.Add(x[i-2,j,s-1] == 0).OnlyEnforceIf(up[i,j,s])
                    # If up, then this value == 0
                    model.Add(x[i,j,s] == 0).OnlyEnforceIf(up[i,j,s])
                    # If up, then the value 1 spot above == 0
                    model.Add(x[i-1,j,s] == 0).OnlyEnforceIf(up[i,j,s])
                    # If up, then the value 2 spots above == 1
                    model.Add(x[i-2,j,s] == 1).OnlyEnforceIf(up[i,j,s])
            

                # Down
                if i <= 4 and any([i not in [3,4],1<j<5]):
                    # If down, then this value is == 1
                    model.Add(x[i,j,s-1] == 1).OnlyEnforceIf(down[i,j,s])  
                    # If down, then the value 1 spot above == 1
                    model.Add(x[i+1,j,s-1] == 1).OnlyEnforceIf(down[i,j,s])                    
                    # If down, then the value 2 spots above == 0
                    model.Add(x[i+2,j,s-1] == 0).OnlyEnforceIf(down[i,j,s])
                    # If down, then this value == 0
                    model.Add(x[i,j,s] == 0).OnlyEnforceIf(down[i,j,s])
                    # If down, then the value 1 spot above == 0
                    model.Add(x[i+1,j,s] == 0).OnlyEnforceIf(down[i,j,s])
                    # If down, then the value 2 spots above == 1
                    model.Add(x[i+2,j,s] == 1).OnlyEnforceIf(down[i,j,s])

                # Left
                if j >= 2 and any([j not in [2,3],1<i<5]):
                    # If left, then this value is == 1
                    model.Add(x[i,j,s-1] == 1).OnlyEnforceIf(left[i,j,s])  
                    # If left, then the value 1 spot left == 1
                    model.Add(x[i,j-1,s-1] == 1).OnlyEnforceIf(left[i,j,s])                    
                    # If left, then the value 2 spots left == 0
                    model.Add(x[i,j-2,s-1] == 0).OnlyEnforceIf(left[i,j,s])
                    # If left, then this value == 0
                    model.Add(x[i,j,s] == 0).OnlyEnforceIf(left[i,j,s])
                    # If left, then the value 1 spot left == 0
                    model.Add(x[i,j-1,s] == 0).OnlyEnforceIf(left[i,j,s])
                    # If left, then the value 2 spots left == 1
                    model.Add(x[i,j-2,s] == 1).OnlyEnforceIf(left[i,j,s])      

                    # Right
                if j <= 4 and any([j not in [3,4],1<i<5]):
                    # If right, then this value is == 1
                    model.Add(x[i,j,s-1] == 1).OnlyEnforceIf(right[i,j,s])  
                    # If right, then the value 1 spot right == 1
                    model.Add(x[i,j+1,s-1] == 1).OnlyEnforceIf(right[i,j,s])                    
                    # If right, then the value 2 spots right == 0
                    model.Add(x[i,j+2,s-1] == 0).OnlyEnforceIf(right[i,j,s])
                    # If right, then this value == 0
                    model.Add(x[i,j,s] == 0).OnlyEnforceIf(right[i,j,s])
                    # If right, then the value 1 spot right == 0
                    model.Add(x[i,j+1,s] == 0).OnlyEnforceIf(right[i,j,s])
                    # If right, then the value 2 spots right == 1
                    model.Add(x[i,j+2,s] == 1).OnlyEnforceIf(right[i,j,s])


                if i-2 >= 0:
                    if grid[i-2][j] in [-2,2]:
                        model.Add(up[i,j,s] == 0)
                if i+2 < len(grid):
                    if grid[i+2][j] in [-2,2]:
                        model.Add(down[i,j,s] == 0)
                if j-2 >= 0:
                    if grid[i][j-2] in [-2,2]:
                        model.Add(left[i,j,s] == 0)
                if j+2 < len(grid):
                    if grid[i][j+2] in [-2,2]:
                        model.Add(right[i,j,s] == 0)

                # if i < 2 or all([i in [2,3],1>=j or j>=5]):
                #     model.Add(up[i,j,s] == 0)
                # if i > 4 or all([i in [3,4],1>=j or j>=5]):
                #     model.Add(down[i,j,s] == 0)
                # if j < 2 or all([j in [2,3],1>=i or i>=5]):
                #     model.Add(left[i,j,s] == 0
                # if j > 4 or all([j in [3,4],1>=i or i>=5]):
                #     model.Add(right[i,j,s] == 0)

status = solver.Solve(model)
print(f"Status = {solver.StatusName(status)}")

np.set_printoptions(threshold=100000)


sol = [solver.value(x[i,j,s]) for s in S for i in n for j in n]
sol = np.reshape(sol, (len(S),7,7))
sol

Status = OPTIMAL


array([[[-2,  2,  1,  1,  1, -2,  2],
        [-2,  2,  1,  1,  1, -2,  2],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  0,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [-2,  2,  1,  1,  1, -2,  2],
        [-2,  2,  1,  1,  1, -2,  2]],

       [[-2,  2,  1,  1,  1, -2,  2],
        [-2,  2,  1,  1,  1, -2,  2],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  0,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [-2,  2,  1,  1,  1, -2,  2],
        [-2,  2,  1,  1,  0, -2,  2]],

       [[-2,  2,  1,  1,  0, -2,  2],
        [-2,  2,  1,  1,  1, -2,  2],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  0,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
        [-2,  2,  1,  1,  1, -2,  2],
        [-2,  2,  1,  1,  0, -2,  2]],

       [[-2,  2,  0,  1,  0, -2,  2],
        [-2,  2,  1,  1,  1, -2,  2],
        [ 1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  0,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1],
      

In [102]:
[(1,i,j) for i in n for j in n if solver.value(down[i,j,1]) == 1]

[(1, 6, 4)]

In [97]:
[(1,i,j) for i in n for j in n if solver.value(left[i,j,1]) == 1]

[]